<h1><strong>Orange fiber Churn Prediction</h1></strong>‏

In [1]:
#Cell No.1

#Importing required libraries 
import pandas as pd
import numpy as np
import pickle 
from sklearn.preprocessing import *
from sklearn.ensemble import * 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import *
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.utils import compute_class_weight
from sklearn.feature_selection import chi2,mutual_info_classif
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,precision_score,roc_auc_score
import warnings
warnings.filterwarnings("ignore")


<h2><strong>Reading and Cleaning data</h2></strong>‏

In [2]:
#Cell No.2

#Reading data file(.xlsx)  
fiber=pd.read_excel('FTTH-DataSet.xlsx')
fiber.head()


,ID,GOVERNORATE,Customer with orange_MONTHS,CUSTOMER_AGE_MONTHS,CUSTOMER_GENDER,COMMITMENT,COMMITMENT_FG,OF_SPEED,OF_PREV_SPEED,MIGRATION_FLAG,...,LAST_POWER_VALIDATION,LAST_LINK_PRIORITY,Disconnection_TOTAL_MAX_day,Disconnection_TOTAL_MIN_day,Disconnection_TOTAL_SUM_Month,Disconnection_TOTAL_MEAN_Month,GB_TOTAL_CONSUMPTION_Month1,GB_TOTAL_CONSUMPTION_Month2,GB_TOTAL_CONSUMPTION_Month3,TARGET
0,1,West Amman,48.741935,567.677419,M,24,1,200,100.0,y,...,Abnormal,Regular,1.0,1.0,32.0,1.000000,645.685532,561.726552,519.477249,0
1,2,West Amman,44.838710,740.580645,M,24,0,100,100.0,y,...,None,Regular,1.0,1.0,4.0,1.000000,174.360611,159.508825,145.229521,0
2,3,West Amman,44.612903,531.096774,M,24,1,200,100.0,y,...,None,Regular,2.0,1.0,8.0,1.333333,299.379466,319.849905,257.353694,0
3,4,Balqa,43.741935,645.612903,M,24,0,200,100.0,y,...,None,Regular,1.0,1.0,6.0,1.000000,477.543451,791.806873,569.299840,0
4,5,West Amman,41.548387,311.774194,M,12,1,400,100.0,y,...,None,Regular,2.0,1.0,16.0,1.142857,891.318951,937.062921,791.050654,0


<h3><strong>Cleaning part</h3></strong>‏

In [3]:
#Cell No.3

#Dropping some columns
fiber=fiber.drop(['ID','LAST_LINK_PRIORITY','LAST_POWER_VALIDATION','CUSTOMER_AGE_MONTHS','CUSTOMER_GENDER','COMMITMENT',
                 'LAST_LINK_QUALITY','LAST_LINK_STATUS','Disconnection_TOTAL_MAX_day','Disconnection_TOTAL_MIN_day',
                 'GB_TOTAL_CONSUMPTION_Month2','GB_TOTAL_CONSUMPTION_Month3','Disconnection_TOTAL_MEAN_Month'],axis=1)


In [4]:
#Cell No.4

#Filling null values with zeros 
cols_with_null_vals=['OF_PREV_SPEED']
for i in cols_with_null_vals:
    fiber[[i]]=fiber[[i]].fillna(0)
fiber[['OF_PREV_SPEED']]= fiber[['OF_PREV_SPEED' ]].fillna(0)
fiber[['GOVERNORATE']]=fiber[['GOVERNORATE']].fillna(method="bfill")
#Manual encoding for Migration flag columns
fiber["MIGRATION_FLAG"]=fiber["MIGRATION_FLAG"].map({"y" :1 , "n" :0})

fiber[['MIGRATION_FLAG']]= fiber[['MIGRATION_FLAG']].fillna(0)


In [5]:
#Cell No.5

#Dropping some rows that have illogical data
fiber.drop(fiber[fiber['GB_TOTAL_CONSUMPTION_Month1']>5000].index, inplace = True)
fiber.drop(fiber[fiber['Disconnection_TOTAL_SUM_Month']>=1000].index, inplace = True)
fiber.dropna(subset=["Disconnection_TOTAL_SUM_Month"],inplace=True)
fiber.isnull().sum()

GOVERNORATE                      0
Customer with orange_MONTHS      0
COMMITMENT_FG                    0
OF_SPEED                         0
OF_PREV_SPEED                    0
MIGRATION_FLAG                   0
Disconnection_TOTAL_SUM_Month    0
GB_TOTAL_CONSUMPTION_Month1      0
TARGET                           0
dtype: int64

<h2><strong>Preparing data</h2></strong>‏

In [6]:
#Cell No.6

#implement a function to find imbalance in our data
def find_imbalanced(name,value1,value2):
    fiber[name].value_counts()
    churners= len(fiber[fiber[name] == value1])
    not_churners = len(fiber[fiber[name] == value2])
    class_distribution_ratio = churners/not_churners
    return class_distribution_ratio

In [7]:
#Cell No.7

find_imbalanced('TARGET',1,0)

0.005357047195585793

<h3><strong>Encoding part</h3></strong>‏

In [8]:
#Cell No.8

#implement a function to convert categorical columns to numeric columns  
def encodeing(encoder_,data,col):
    label_encoder= encoder_
    data[col]=label_encoder.fit_transform(data[col])
    return data


In [9]:
#Cell No.9

encoder =pd.get_dummies(fiber['GOVERNORATE'])
data1=pd.concat([fiber,encoder],axis=1)
data1=data1.drop(['GOVERNORATE'],axis=1)


<h3><strong>Fixing imbalnce</h3></strong>‏

In [10]:
#Cell No.10

#spliting function 
def split(data,col):
    X=data.drop(col,axis=1)
    y=data[col]
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25, stratify=y  ,random_state=42)
    
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test=split(data1,'TARGET') 

In [11]:
#Cell No.11

#imbalanced fix method  
weights = compute_class_weight('balanced', classes=y_train.unique(), y=y_train)
weights=dict(enumerate(weights))

<h3><strong>Feature selection</h3></strong>‏

In [12]:
#Cell No.12

#Choose best features using SelectKbest()
sel_eight_cols = SelectKBest(chi2, k=8)
new=sel_eight_cols.fit_transform(X_train, y_train)
X_train.columns[sel_eight_cols.get_support()]

Index(['Customer with orange_MONTHS', 'COMMITMENT_FG', 'OF_SPEED',
       'OF_PREV_SPEED', 'MIGRATION_FLAG', 'Disconnection_TOTAL_SUM_Month',
       'GB_TOTAL_CONSUMPTION_Month1', 'West Amman'],
      dtype='object')

<h1><strong>Modeling part</h1></strong>‏


In [13]:
#Cell No.13

# Running initial model
def modeling(model):
  modl= model 
  modl.fit(X_train,y_train) 
  ypred= modl.predict(X_test)
  print('accuracy for test :',round(accuracy_score(y_test, ypred),2))
  print('accuracy for train :',round(accuracy_score(y_train, modl.predict(X_train)),2))
  return 



<h3><strong>Algorethim testing</h3></strong>‏

In [14]:
#Cell No.14

#Test these algorethims using deafult Hyperparameter tuning 
classifiers = {
    "LogisiticRegression": LogisticRegression(random_state=0,class_weight=weights),
    "RandomForestClassifier": RandomForestClassifier(random_state=0,class_weight=weights)}

for i,j in classifiers.items():
    print(i)
    print(modeling(j))

LogisiticRegression
accuracy for test : 0.82
accuracy for train : 0.82
None
RandomForestClassifier
accuracy for test : 0.99
accuracy for train : 1.0
None


<h3><strong>Hyperparameter selection</h3></strong>‏

In [15]:
#Cell No.15

#set a parameter grid values for logistic regression and random forest 
parameters_clf = [{'solver': ['newton-cg', 'liblinear']},
              {'penalty':[ 'elasticnet', 'l1']},
              {'C':[0.001, 0.01, 0.1, 1, 10, 100]
               ,'max_iter':[500,1000,1500]}]
param_grid_random = { 
    'n_estimators': [50,100,150,200,250],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [2,4,5,6,7,8],
    'criterion' :['gini', 'entropy']}

In [16]:
#Cell No.16

#set a function to get the parameters for each algorethims using grid search
def grid_search(model,parameters,scoring):
    grid=GridSearchCV(estimator=model, param_grid=parameters, cv= 5,n_jobs=-1,scoring=scoring)
    grid.fit(X_train, y_train)
    return grid.best_params_


<h3><strong>Running Grid search to get best parameters for both algorethims</h3></strong>




In [17]:
#Cell No.17

grid_search(LogisticRegression(random_state=0,),parameters_clf,'f1')

In [18]:
#Cell No.18

grid_search(RandomForestClassifier(random_state=0), param_grid_random, 'f1')

In [19]:
#Cell No.19

#Comparing reasults between algorethims after using grid search
classifiers = {
    "LogisiticRegression": LogisticRegression(solver='newton-cg' ,class_weight=weights),
    "RandomForestClassifier": RandomForestClassifier(criterion='gini', max_depth=6,max_features='auto',n_estimators=50,random_state=0,class_weight=weights)}

#for loop to print the results from the modeling function
for i,j in classifiers.items():
    print(i)
    print(modeling(j))


LogisiticRegression
accuracy for test : 0.82
accuracy for train : 0.82
None
RandomForestClassifier
accuracy for test : 0.83
accuracy for train : 0.83
None


<h3><strong>Final model</h3></strong>‏


In [20]:
#Cell No.20

#Best model we chose in our project
model= RandomForestClassifier(criterion='gini', max_depth=6,max_features='auto',n_estimators=100,random_state=0)
model.fit(X_train,y_train)

RandomForestClassifier(max_depth=6, random_state=0)

<h3><strong>User interface</h3></strong>‏


In [21]:
#Cell No.21

#Create a pickel file has the best model to use it in notebook (2)
filename ="trained_model.pkl"
pickle.dump(model,open(filename,"wb"))
load_model=pickle.load(open("trained_model.pkl","rb"))
predictions = load_model.predict(X_test)

